In [1]:
# Importing useful libraries
import time
import random
import requests 

# Creating an empty global dictionary to store the leaderboard data (usernames, games, scores)
leaderboard_dict = {}


# Defining a function to get the username of the user, when a single-player game is chosen
def get_single_username():
    player = input("Enter your username: ")
    return player


# Defining a function to get two usernames from the users, when a dual-player game is chosen
def get_two_usernames():
    username1 = input("Enter the first username: ")
    username2 = input("Enter the second username: ")
    return [username1, username2]


# Defining a function to display the leaderboard for a specific game
# If no specific game is past as an argument, show the leaderboard for all games
def display_leaderboard(game_name=None):
    if game_name is None:

        if game_name is None:
            print("\nLeaderboard:")

        # Collecting the set of all games played by any player 
        # From the leaderboard dict first all values are extracted, then it is iterated over the game names for each 
        # statistics in the dict to create an iterable of all the game names, which is then converted to a set
        all_games = set(game for stats in leaderboard_dict.values() for game in stats.keys())
        
        # Initiating the leaderboard for every game through a for loop
        for game in all_games:
            print(f"\n{game}:")

            # Sorting players by wins for the current game, and for those with the same wins, by games played (descending)
            players_sorted = sorted(leaderboard_dict.items(),
                                    key=lambda x, current_game=game: (-x[1].get(current_game, {'wins': 0})['wins'],
                                                                      -x[1].get(current_game, {'games_played': 0})[
                                                                          'games_played']))
                    
            # Displaying the sorted leaderboard
            for player, stats in players_sorted:
                game_stats = stats.get(game, {'wins': 0, 'games_played': 0})
                print(f"  {player}: {game_stats['wins']} win(s) out of {game_stats['games_played']} game(s) played")

    else:
        # Checking if the specific game past as argument has no data available in the leaderboard dictionary
        if all(game_name not in stats for _, stats in leaderboard_dict.items()):
            print(f"No data available for {game_name}.")
            return

        # Initiating the leaderboard for the game in case there is data in the leaderboard
        print(f"Leaderboard for {game_name}:")

        # Sorting players for a specific game by wins and then games played in descending order
        players_sorted = sorted(leaderboard_dict.items(),
                                key=lambda x: (-x[1].get(game_name, {'wins': 0})['wins'],
                                               -x[1].get(game_name, {'games_played': 0})['games_played']))

        # Displaying the sorted leaderboard for the specified game
        for player, stats in players_sorted:
            game_stats = stats.get(game_name, {'wins': 0, 'games_played': 0})
            print(f"  {player}: {game_stats['wins']} win(s) out of {game_stats['games_played']} game(s) played")


# Defining a function to update the leaderboard for a specified game and user
def update_leaderboard(username, game_name, win):
    # Initializing player's entry if the username is not present in the dictionary
    if username not in leaderboard_dict:
        leaderboard_dict[username] = {}

    # Initializing game entry if the game is not present for that player
    if game_name not in leaderboard_dict[username]:
        leaderboard_dict[username][game_name] = {'games_played': 0, 'wins': 0}

    # Updating the number of games played 
    leaderboard_dict[username][game_name]['games_played'] += 1

    # Updating the number of wins if the player won
    if win:
        leaderboard_dict[username][game_name]['wins'] += 1


# Defining a function to play the Hangman game
def play_hangman(username):
    print(f"\nWelcome to the Hangman game, {username}!")
    print(
        "Attempt to guess the hidden word by guessing letters. If you make 10 incorrect guesses, you lose. "
        "Unveil the hidden word before running out of attempts!")
    print()


    # Providing the API information. The API is used to generate random words
    api_url = 'https://api.api-ninjas.com/v1/randomword'
    response = requests.get(api_url,
                            headers={'X-Api-Key': 'Kh352YamABDawfc9xo2NdA==dihjciOjxZzyqixl'})  

    if response.status_code == requests.codes.ok:
        # Extracting the word from the API's response JSON
        word_data = response.json()
        word = word_data.get("word", "").lower()
    else:
        print("Error:", response.status_code, response.text)
        exit()  # Exiting the program if there's an error when retrieving the word

    guessed_characters = set()  # Creating an empty set to store guessed characters
    guesses = ''  # Creating a variable with an empty value for guesses
    turns = 10  # Determining the number of turns

    # Creating a while loop to control the game's turns
    while turns > 0:
        failed = 0  # Creating a counter for incorrect guesses

        # Iterating through each character in the generated word and checking whether the player has guessed it
        # At the same time, displaying the right characters and the empty charachters of the word
        # At the same time, counting the number of incorrect guesses
        for char in word:
            if char in guesses:
                print(char, end="")
            else:
                print("_ ", end="")
                failed += 1

        # Checking if the player has guessed all the letters in the word
        if failed == 0:
            print("\nCongratulations, you won!")
            update_leaderboard(username, "Hangman", True)
            return 1  # Win

        guess = input("\nGuess a letter: ").lower()

        # Checking that the player enters a single letter
        if len(guess) != 1 or not guess.isalpha():
            print("Please enter a single letter.")
            print()
            continue

        # Checking that the player doesn't guess the same charachter twice    
        if guess in guessed_characters:
            print("You have already guessed that letter.")
            print()
            continue

        guessed_characters.add(guess)
        guesses += guess

        # Controlling the game if the guess is wrong
        if guess not in word:
            turns -= 1
            print("Wrong!")
            print(f"You have {turns} more guesses")
            print()

            if turns == 0:
                print(f"You lose. The correct word was '{word}'.")
                update_leaderboard(username, "Hangman", False)
                return -1  # Loss

    return 1  # Win


# Defining a function to play the 'Higher or Lower' card game
def play_higher_lower(username):
    print(f"\nWelcome to higher or lower, {username}!")
    print("Guess whether the next card drawn will be 'higher' or 'lower' than the current card. "
          "The game is won by making 8 consecutive correct predictions!")

    # Defining the ranks and suits of a standard deck of playing cards
    ranks = ["Ace", "2", "3", "4", "5", "6", "7", "8", "9", "10", "Jack", "Queen", "King"]
    suits = ["Clubs", "Hearts", "Diamonds", "Spades"]

    # Creating and shuffling the deck
    deck = []
    value = 1
    for rank in ranks:
        for suit in suits:
            deck.append([rank + " of " + suit, value])  # Each card has a name and a numeric value
        value += 1
    random.shuffle(deck)

    # Initializing the game variables
    score = 0  # Player's score
    consecutive_correct_guesses = 0  # Counter for consecutive correct guesses
    card1 = deck.pop(0)  # Draw the first card

    # Main game loop
    while True:
        print("Your score so far is", score)
        print("\nThe current card is", card1[0])

        # Getting the user's input for higher or lower
        while True:
            choice = input("higher or lower? ").lower()
            if choice in ["h", "higher", "l", "lower"]:
                break  # Accepting both single letters and full words

        # Drawing the next card
        card2 = deck.pop(0)
        print("The next card picked is", card2[0])
        time.sleep(1)  # Pause for a moment to build suspense

        # Determine and announce if the user's guess was correct
        correct_guess = ((choice[0] == "h" and card2[1] > card1[1]) or
                         (choice[0] == "l" and card2[1] < card1[1]))
        if correct_guess:
            print("Correct!")
            score += 1 # Updating the various counters
            consecutive_correct_guesses += 1
            if consecutive_correct_guesses == 8: # Keeping track of when the user wins
                print("Congratulations! You've made 8 consecutive correct guesses. You win!")
                update_leaderboard(username, "Higher-Lower", True)  # Updating the leaderboard
                return 1  # Win
        else:
            print("Wrong!")
            update_leaderboard(username, "Higher-Lower", False)  # Updating the leaderboard
            return -1  # Loss

        # Updating the current card for the next round
        card1 = card2


# Defining a function to play a game of Tic Tac Toe between two players
def play_tic_tac_toe(player1_name, player2_name):
    print(f"\nWelcome to tic tac toe, {player1_name} and {player2_name}!")
    print("Place your X or O strategically on the grid. Form a horizontal, vertical, "
          "or diagonal line with three of your marks to win. Outsmart your opponent and claim victory!")

    def print_board(board, show_numbers=False):
        """Print the Tic Tac Toe board with optional position numbers."""
        for i, row in enumerate(board):
            print("|".join(str(cell) if cell != ' ' else (str(3 * i + j + 1) if show_numbers else ' ')
                           for j, cell in enumerate(row)))
            if i < 2:
                print("-" * 5)

    def print_guide_board():
        """Print a guide board with numbered positions for reference."""
        guide_board = [[' ' for _ in range(3)] for _ in range(3)]
        print_board(guide_board, show_numbers=True)

    def check_winner(board):
        """Check if there is a winner in the current board."""
        # Check rows, columns, and diagonals for a win
        for row in board:
            if row[0] == row[1] == row[2] != ' ':
                return True

        for col in range(3):
            if board[0][col] == board[1][col] == board[2][col] != ' ':
                return True

        if board[0][0] == board[1][1] == board[2][2] != ' ' or board[0][2] == board[1][1] == board[2][0] != ' ':
            return True

        return False

    def is_board_full(board):
        """Check if the board is full and no more moves can be made."""
        return all(cell != ' ' for row in board for cell in row)

    def get_player_move(player):
        """Get a valid move from the player."""
        while True:
            try:
                move = int(input(f"{player}, enter your move (1-9): "))
                if 1 <= move <= 9:
                    return divmod(move - 1, 3)
                # Making sure that the input is valid
                else:
                    print("Invalid input. Please enter a number between 1 and 9.")
            except ValueError:
                print("Invalid input. Please enter a number.")

    # Initializing an empty 3x3 board
    board = [[' ' for _ in range(3)] for _ in range(3)]
    current_player = player1_name
    show_numbers = True

    # Displaying the initial guide board with numbers by calling the previously defined function
    print()
    print("Here's the empty board with numbered positions:")
    print_guide_board()
    time.sleep(4) # Pausing the program for 4 seconds to allow the user to choose their move

    while True:
        # Displaying the current state of the board by calling the previously defined function
        print("\nCurrent Board:")
        print_board(board)

        # Getting the current player's move by calling the previously defined function
        row, col = get_player_move(current_player)

        # Checking if the cell is already taken
        if board[row][col] == ' ':
            board[row][col] = 'X' if current_player == player1_name else 'O'
            show_numbers = False

            # Checking for a win or tie and directly updating the leaderboard with the previously defined function
            if check_winner(board):
                print_board(board)
                winner = current_player
                loser = player2_name if current_player == player1_name else player1_name
                print(f"{winner} wins!")
                update_leaderboard(winner, "Tic Tac Toe", True)  # Updating winner's data
                update_leaderboard(loser, "Tic Tac Toe", False)  # Updating loser's data
                break
            elif is_board_full(board):
                print_board(board)
                print("It's a tie!")
                # Updating both players' games played stats
                update_leaderboard(player1_name, "Tic Tac Toe", False)
                update_leaderboard(player2_name, "Tic Tac Toe", False)
                break

            # Switching players
            current_player = player2_name if current_player == player1_name else player1_name
        else:
            print("Cell already taken. Try again.")


# Defining the main function of the game console. It presents the options to the player and handles their choices
def main():
    print("Welcome to the Game Cube!")
    global leaderboard_dict  # Access the global leaderboard dictionary

    while True:
        time.sleep(2)
        print("\nOptions:")
        print("1: Hangman")
        print("2: Higher or Lower")
        print("3: Tic Tac Toe (2-player)")
        print("4: Display Leaderboard")
        print("5: Quit Game Cube")

        choice = input("Enter the number of your choice: ")

        # Processing the player's choice: getting their username, playing the game, and displaying the username
        # All is done by calling the previously defined functions
        if choice == '1':
            # Play Hangman
            player1 = get_single_username()
            play_hangman(player1)
            display_leaderboard("Hangman")

        elif choice == '2':
            # Play Higher or Lower
            player1 = get_single_username()
            play_higher_lower(player1)
            display_leaderboard("Higher-Lower")

        elif choice == '3':
            # Play Tic Tac Toe
            usernames = get_two_usernames()
            play_tic_tac_toe(*usernames)
            display_leaderboard("Tic Tac Toe")

        elif choice == '4':
            # Display the leaderboard
            display_leaderboard()

        elif choice == '5':
            # Exit the game cube and reset leaderboard
            leaderboard_dict = {}  # Resetting the leaderboard
            print("Goodbye, hope to see you soon!")
            break

        else:
            # Handling invalid choices
            print()
            print("Invalid choice. Please enter a number between 1 and 5.")


if __name__ == "__main__":
    main()

main()



Welcome to the Game Cube!

Options:
1: Hangman
2: Higher or Lower
3: Tic Tac Toe (2-player)
4: Display Leaderboard
5: Quit Game Cube


KeyboardInterrupt: Interrupted by user

# 